In [ ]:
# %pip install pysm3 pixell healpy

In [ ]:
import pysm3
from matplotlib import pyplot as plt
import numpy as np
import pixell
from pysm3 import units as u
import healpy as hp

In [ ]:
nside = 256  # higher for high resolution
healpix_reso = hp.nside2resol(nside) * u.radian
healpix_reso.to(u.arcmin)

In [ ]:
npix = hp.nside2npix(nside)
print(npix / 1e6, "Mpix")

In [ ]:
seed = 1

In [ ]:
dust = pysm3.ModifiedBlackBodyRealization(
    nside=nside,
    amplitude_modulation_temp_alm="dust_gnilc/raw/gnilc_dust_temperature_modulation_alms_lmax768.fits.gz",
    amplitude_modulation_pol_alm="dust_gnilc/raw/gnilc_dust_polarization_modulation_alms_lmax768.fits.gz",
    largescale_alm="dust_gnilc/raw/gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax1024_complex64.fits.gz",
    small_scale_cl="dust_gnilc/raw/gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz",
    largescale_alm_mbb_index="dust_gnilc/raw/gnilc_dust_largescale_template_beta_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_index="dust_gnilc/raw/gnilc_dust_small_scales_beta_cl_lmax16384_2023.06.06.fits.gz",
    largescale_alm_mbb_temperature="dust_gnilc/raw/gnilc_dust_largescale_template_Td_alm_nside2048_lmax1024.fits.gz",
    small_scale_cl_mbb_temperature="dust_gnilc/raw/gnilc_dust_small_scales_Td_cl_lmax16384_2023.06.06.fits.gz",
    freq_ref="353 GHz",
    seeds=[seed, seed + 1000, seed + 2000],
    max_nside=8192,
)

In [ ]:
sky = pysm3.Sky(nside=nside, component_objects=[dust])

In [ ]:
freq = 353 * u.GHz

In [ ]:
m = sky.get_emission(freq)

In [ ]:
hp.mollview(m[0], max=1e3)

In [ ]:
import pixell.enmap

In [ ]:
import numpy as np

import astropy.units as u

car_reso = (np.pi / np.round(np.pi / healpix_reso.value)) * u.radian
print(healpix_reso.to(u.arcmin), car_reso.to(u.arcmin))

In [ ]:
m_car = pysm3.apply_smoothing_and_coord_transform(
    m, output_car_resol=car_reso, return_healpix=False, return_car=True
)

In [ ]:
m_car.shape

In [ ]:
from pixell.enplot import plot, pshow

In [ ]:
pshow(m_car)

The maps have Intensity and then the Q and U components of polarization, we can start just using Intensity

In [ ]:
import h5py

with h5py.File(f"d11_dust_{seed:05d}.h5", "w") as f:
    f.create_dataset("I", data=m_car[0])

In [ ]:
m_car[0]

In [ ]:
m_car[0] - h5py.File("d11_dust_00000.h5")["I"][0]